In [165]:
import numpy as np
import pandas as pd
import sys
sys.path
sys.path.append('../app')
import calcul
import importlib
import plotly.express as px

In [242]:
data = {
    'fridge': {
        'presence': True,
        'category': 'F',
        'power_kW': 2,
        'min_h': 6,
        'max_h': 8
    },
    'freezer': {
        'presence': False,
        'category': 'F',
        'power_kW': 2.5,
        'min_h': 6,
        'max_h': 8
    },
    'washing_machine': {
        'presence': True,
        'category': 'A',
        'power_kW': 1.5,
        'min_h': 1,
        'max_h': 4
    },
    'dishwasher': {
        'presence': False,
        'category': 'A',
        'power_kW': 2.5,
        'min_h': 1,
        'max_h': 4
    },
    'induction_stove': {
        'presence': False,
        'category': 'A',
        'power_kW': 3,
        'min_h': 1,
        'max_h': 4
    },
    'tv': {
        'presence': True,
        'category': 'L',
        'power_kW': .5,
        'min_h': 4,
        'max_h': 24
    },
    'small_light': {
        'presence': True,
        'category': 'L',
        'power_kW': .1,
        'min_h': 4,
        'max_h': 24
    },
    'big_light': {
        'presence': True,
        'category': 'L',
        'power_kW': .8,
        'min_h': 4,
        'max_h': 24
    },
}

In [243]:
df = calcul.build_base_df(data)

print(f"""Regarding the appliance list, your consumption is expected to be at least \
{df['min_kWh'].sum()} kWh and at most {df['max_kWh'].sum()} kWh""")

df

Regarding the appliance list, your consumption is expected to be at least 19.1 kWh and at most 55.6 kWh


,presence,category,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,ratio_to_avg,ratio_to_max,ratio_to_min
fridge,True,F,2,6,8,12,14.0,16,0.374833,0.28777,0.628272
freezer,False,F,2.5,6,8,0.0,0.0,0.0,0.0,0.0,0.0
washing_machine,True,A,1.5,1,4,1.5,3.75,6.0,0.100402,0.107914,0.078534
dishwasher,False,A,2.5,1,4,0.0,0.0,0.0,0.0,0.0,0.0
induction_stove,False,A,3,1,4,0,0.0,0,0.0,0.0,0.0
tv,True,L,0.5,4,24,2.0,7.0,12.0,0.187416,0.215827,0.104712
small_light,True,L,0.1,4,24,0.4,1.4,2.4,0.037483,0.043165,0.020942
big_light,True,L,0.8,4,24,3.2,11.2,19.2,0.299866,0.345324,0.167539


In [244]:
calcul.bargraph(df, 'avg_kWh', 'Average consumption')

min:       19.1 kWh
avg:       37.35 kWh
max:       55.6 kWh
input:     no input yet kWh
        


## Measured consumption input

In [245]:
e_tot = 19

## Frist estimation
- First estimation of the consumion based on the average ratios

In [246]:
df = calcul.first_estimation(df, e_tot)
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,12.000000,6.000000
freezer,2.5,6,8,0.0,0.00,0.0,0.000000,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,1.907631,1.271754
dishwasher,2.5,1,4,0.0,0.00,0.0,0.000000,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.000000,0.000000
tv,0.5,4,24,2.0,7.00,12.0,3.560910,7.121821
small_light,0.1,4,24,0.4,1.40,2.4,0.712182,7.121821
big_light,0.8,4,24,3.2,11.20,19.2,5.697456,7.121821


In [247]:
bargraph(df, e_tot=e_tot)

min:       19.1 kWh
avg:       37.35 kWh
max:       55.6 kWh
estimated: 23.87817938420348 kWh
input:     19 kWh
        


## Correction fo first estimation :
- Problem : due to the high consumption of appliance of type 'F', the total estimation can be above the measured input consumption.
- Solution, concerned appliance are isolated and new estimation is calculated on the other appliances
- The problem can happen a 2nd time with type A appliances

In [248]:
if not calcul.test_esti(df):
    print(1)
    df_1, df_2, e_tot_residual = calcul.estimation_correction_type_1(df, e_tot)
    if not calcul.test_esti(df_2):
        df_21, df_22, e_tot_residual = calcul.estimation_correction_type_1(df_2, e_tot_residual)
        df = pd.concat([df_1, df_21, df_22])
    else:
        df = pd.concat([df_1, df_2])

if not calcul.test_esti_2(df):
    print(2)
    df_1, df_2, e_tot_residual = calcul.estimation_correction_type_1_bis(df, e_tot)
    if not calcul.test_esti(df_2):
        df_21, df_22, e_tot_residual = calcul.estimation_correction_type_1_bis(df_2, e_tot_residual)
        df = pd.concat([df_1, df_21, df_22])
    else:
        df = pd.concat([df_1, df_2])

1


In [249]:
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,12.0,6.000000
freezer,2.5,6,8,0.0,0.00,0.0,0.0,0.000000
dishwasher,2.5,1,4,0.0,0.00,0.0,0.0,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.0,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,1.5,1.271754
tv,0.5,4,24,2.0,7.00,12.0,2.0,7.121821
small_light,0.1,4,24,0.4,1.40,2.4,0.4,7.121821
big_light,0.8,4,24,3.2,11.20,19.2,3.2,7.121821


In [250]:
bargraph(df, e_tot=e_tot)

min:       19.1 kWh
avg:       37.35 kWh
max:       55.6 kWh
estimated: 19.1 kWh
input:     19 kWh
        


## Second correction
- If the measured consumption is really high, close to the maximum (resp.minimum), then estimations are calculated base on the maximum (resp. minimum) ratios instead of the averaged ratios.
ex : 

{
    'fridge': {
        'presence': True,
    },
    'freezer': {
        'presence': False,
    },
    'washing_machine': {
        'presence': True,
    },
    'dishwasher': {
        'presence': False,
    },
    'induction_stove': {
        'presence': False,
    },
    'tv': {
        'presence': True,
    },
    'small_light': {
        'presence': True,
    },
    'big_light': {
        'presence': False,
    },
}
e_tot = 36

In [251]:
df = calcul.estimation_correction_type_2(df, e_tot)

Correction because overestimating


In [252]:
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,11.937173,5.968586
freezer,2.5,6,8,0.0,0.00,0.0,0.000000,0.000000
dishwasher,2.5,1,4,0.0,0.00,0.0,0.000000,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.000000,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,1.492147,0.994764
tv,0.5,4,24,2.0,7.00,12.0,1.989529,3.979058
small_light,0.1,4,24,0.4,1.40,2.4,0.397906,3.979058
big_light,0.8,4,24,3.2,11.20,19.2,3.183246,3.979058


In [253]:
bargraph(df, e_tot=e_tot)

min:       19.1 kWh
avg:       37.35 kWh
max:       55.6 kWh
estimated: 19.0 kWh
input:     19 kWh
        


In [155]:
fig = px.bar(df, x=df.index, y='esti_kWh')
fig.show()